# Required imports

In [1]:
import pandas as pd

# Load the dataset

In [2]:
df = pd.read_pickle("../datasets/10kSongs_1.pickle")

# Explore the data

In [3]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 54 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   analysis_sample_rate        10000 non-null  int32  
 1   artist_7digitalid           10000 non-null  int32  
 2   artist_familiarity          10000 non-null  float64
 3   artist_hotttnesss           10000 non-null  float64
 4   artist_id                   10000 non-null  object 
 5   artist_latitude             10000 non-null  float64
 6   artist_location             10000 non-null  object 
 7   artist_longitude            10000 non-null  float64
 8   artist_mbid                 10000 non-null  object 
 9   artist_mbtags               10000 non-null  object 
 10  artist_mbtags_count         10000 non-null  object 
 11  artist_name                 10000 non-null  object 
 12  artist_playmeid             10000 non-null  int32  
 13  artist_terms                1000

If we want to extract the df as a compressed json:

In [21]:
df.to_json("20kSongs.json.gz", orient="records", compression={'method': 'gzip', 'compresslevel': 9, 'mtime': 1})

If we want to extract the df to a pickle:

In [16]:
df.to_pickle("20kSongs.pickle.gz", compression={'method': 'gzip', 'compresslevel': 9, 'mtime': 1})

Test whether we can read the pickle we've just created:

In [17]:
df_extracted = pd.read_pickle("20kSongs.pickle.gz", compression={'method': 'gzip', 'compresslevel': 9, 'mtime': 1})